In [ ]:
# Importing libraries
import json
import numpy as np

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Dense,ReLU
from tensorflow.keras import Sequential

In [1]:
import json
# Load json data
with open("traindata.json","r") as file:
    Data = json.load(file)    

In [ ]:
# Placeholder for storing values
all_patterns = []
tags = []
xy = []

Processing Data

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return " ".join(filtered_tokens)

In [ ]:
for question in Data['questions']:
    tag = question['tag']
    tags.append(tag)
    
    for pattern in question['keywords']:
        all_patterns.append(pattern)
        xy.append((pattern,tag))

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(all_patterns)

In [ ]:
patterns = tokenizer.texts_to_sequences(all_patterns)

In [ ]:
max_pattern_lenght = len(all_patterns)
sequences = pad_sequences(patterns, maxlen=max_pattern_lenght)

In [ ]:
label_to_index = {tag:i for i,tag in enumerate(set(tags))}

In [ ]:
labels = np.array([label_to_index[label[1]] for label in xy], dtype=np.int32)

In [ ]:
def create_neural_net(input_size,hidden_size, num_classess):
    model = Sequential()
    model.add(Dense(hidden_size,input_shape=(input_size,),activation='relu'))
    model.add(Dense(hidden_size,activation='relu'))
    model.add(Dense(num_classess,activation='softmax'))
    
    return model

In [ ]:
Model = create_neural_net(len(sequences[0]),8,3)

In [ ]:
Model.summary()

In [ ]:
Model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

In [ ]:
Model.fit(sequences, labels, epochs=1200, batch_size=4, callbacks=[early_stopping])

In [ ]:
post = "Im trying to install anaconda, I have but my conda is not working"
post = preprocess_text(post)
post_sentence = tokenizer.texts_to_sequences([post])
print(post_sentence)
post_sentence_padded = pad_sequences(post_sentence,maxlen=max_pattern_lenght)
print(post_sentence_padded)

In [ ]:
predicted_probs = Model.predict(post_sentence_padded)
predicted_label_index = np.argmax(predicted_probs)
predicted_tag = [tag for tag, index in label_to_index.items() if index == predicted_label_index][0]

print("Predicted tag:", predicted_tag)

In [2]:
from Utils.utils import *

In [3]:
make = preprocess_text("Hello world this is fiifi on the mick")
print(make)

hello world fiifi mick


In [4]:
bag = bag_of_words("kofi said hello to you all",["hello","said"],{"hello":0,"said":1})
print(bag)

[1. 1.]


In [5]:
tags,all_patterns,xy = processDataSet(Data)